In [60]:
%pip install lomas-client


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [61]:
# Step 1
from lomas_client import Client
import numpy as np
import opendp.prelude as dp

# Step 2
APP_URL = "https://user-stuartbenoliel-67556-lomas-server-user.lab.sspcloud.fr"
# URL mort
USER_NAME = "Dr. Antartica"
DATASET_NAME = "PENGUIN"
client = Client(url=APP_URL, user_name = USER_NAME, dataset_name = DATASET_NAME)

# Step 3

In [62]:
penguin_metadata = client.get_dataset_metadata()
penguin_metadata

{'max_ids': 1,
 'rows': 344,
 'row_privacy': True,
 'censor_dims': False,
 'columns': {'species': {'private_id': False,
   'nullable': False,
   'max_partition_length': None,
   'max_influenced_partitions': None,
   'max_partition_contributions': None,
   'type': 'string',
   'cardinality': 3,
   'categories': ['Adelie', 'Chinstrap', 'Gentoo']},
  'island': {'private_id': False,
   'nullable': False,
   'max_partition_length': None,
   'max_influenced_partitions': None,
   'max_partition_contributions': None,
   'type': 'string',
   'cardinality': 3,
   'categories': ['Torgersen', 'Biscoe', 'Dream']},
  'bill_length_mm': {'private_id': False,
   'nullable': False,
   'max_partition_length': None,
   'max_influenced_partitions': None,
   'max_partition_contributions': None,
   'type': 'float',
   'precision': 64,
   'lower': 30.0,
   'upper': 65.0},
  'bill_depth_mm': {'private_id': False,
   'nullable': False,
   'max_partition_length': None,
   'max_influenced_partitions': None,
   'm

In [63]:
NB_PENGUINS = penguin_metadata["rows"]
columns = list(penguin_metadata["columns"].keys())

df_dummy = client.get_dummy_dataset()

print(df_dummy.shape)
df_dummy.head()

(100, 7)


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Dream,61.800324,20.774048,227.899635,3509.636957,FEMALE
1,Gentoo,Torgersen,54.489750,22.718264,163.455221,6592.209478,FEMALE
2,Chinstrap,Dream,39.305449,18.007412,203.606804,5906.470177,FEMALE
3,Chinstrap,Torgersen,63.921173,14.438975,201.422287,2552.942055,FEMALE
4,Chinstrap,Dream,57.256282,13.139363,235.757214,6985.173289,MALE


In [64]:
print(client.get_initial_budget())
print(client.get_total_spent_budget())
print(client.get_remaining_budget())

initial_epsilon=10.0 initial_delta=0.005
total_spent_epsilon=0.0 total_spent_delta=0.0
remaining_epsilon=10.0 remaining_delta=0.005


In [65]:
num_rows_pipeline = (
    dp.t.make_split_dataframe(separator=",", col_names=columns) >>
    dp.t.make_select_column(key="species", TOA=str) >>
    dp.t.then_count() >>
    dp.m.then_laplace(scale=5.) # scale arbitrary
)

num_rows = client.opendp.query(
    opendp_pipeline = num_rows_pipeline,
    dummy = True
).result.value

num_rows

98

In [78]:
bill_length_min = penguin_metadata['columns']['bill_length_mm']['lower']
bill_length_max = penguin_metadata['columns']['bill_length_mm']['upper']
bill_length_min, bill_length_max

bill_length_pipeline = (
    dp.t.make_split_dataframe(separator=",", col_names=columns) >>
    dp.t.make_select_column(key="bill_length_mm", TOA=str) >>
    dp.t.then_cast_default(TOA=float) >>
    dp.t.then_clamp(bounds=(bill_length_min, bill_length_max)) >>
    dp.t.then_resize(size=NB_PENGUINS, constant=40.0) >>
    dp.t.then_mean() >>
    dp.m.then_gaussian(scale=0.5) # Noise addition mechanism instructions
)

In [77]:
cost_res = client.opendp.cost(
    opendp_pipeline = bill_length_pipeline,
    fixed_delta=1e-5
)
cost_res

CostResponse(epsilon=0.20348837209353815, delta=0.0)

In [22]:
dummy_mean_res = client.opendp.query(
    opendp_pipeline = bill_length_pipeline,
    dummy=True,
    fixed_delta=1e-5
)
print(f"Dummy result for mean: {np.round(dummy_mean_res.result.value, 2)}")

Dummy result for mean: 42.26


In [ ]:
mean_res = client.opendp.query(
    opendp_pipeline = bill_length_pipeline,
    fixed_delta=1e-5
)

mean_bill_length = np.round(mean_res.result.value, 2)
print(f"Mean of bill length: {mean_bill_length} (from opendp query).")

ValidationError: 1 validation error for tagged-union[InvalidQueryExceptionModel,ExternalLibraryExceptionModel,UnauthorizedAccessExceptionModel,InternalServerExceptionModel]
  JSON input should be string, bytes or bytearray [type=json_type, input_value={'type': 'InvalidQueryExc...lta remaining 0.00498.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/json_type

In [ ]:
previous_queries = client.get_previous_queries()
len(previous_queries)

5

In [ ]:
# OpenDP
var_bill_length_query = previous_queries[0]
var_bill_length_query

{'user_name': 'Dr. Antartica',
 'dataset_name': 'PENGUIN',
 'dp_librairy': 'opendp',
 'client_input': {'dataset_name': 'PENGUIN',
  'opendp_json': Measurement(
      input_domain   = AtomDomain(T=String),
      input_metric   = SymmetricDistance(),
      output_measure = MaxDivergence(f64)),
  'fixed_delta': None},
 'response': {'epsilon': 0.7122093023265228,
  'delta': 0.0,
  'requested_by': 'Dr. Antartica',
  'result': {'res_type': 'opendp', 'value': 21.587582284159083}},
 'timestamp': 1744791577.182293}

In [ ]:
categories = penguin_metadata['columns']['species']['categories']
categories

['Adelie', 'Chinstrap', 'Gentoo']

In [ ]:
species_count_pipeline = (
    dp.t.make_split_dataframe(separator=",", col_names=columns) >>
    dp.t.make_select_column(key="species", TOA=str) >>
    dp.t.then_count_by_categories(categories=categories) >>
    dp.m.then_laplace(scale=0.5)
)

dummy_res = client.opendp.query(
    opendp_pipeline = species_count_pipeline, 
    dummy=True
)

for i, count in enumerate(dummy_res.result.value):
    if i == len(categories):
        print(f"Species Unknown has {count} penguins.")
    else:
        print(f"Species {categories[i]} has {count} penguins.")

Dummy result for histogram: [27, 33, 41, 0]


In [ ]:
cost_res = client.opendp.cost(
    opendp_pipeline = species_count_pipeline
)
cost_res

CostResponse(epsilon=2.0, delta=0.0)

In [ ]:
# Import library
import numpy as np
import pandas as pd
import polars as pl
import opendp.prelude as dp

USER_NAME = "Dr. FSO"
DATASET_NAME = "FSO_INCOME_SYNTHETIC"
client = Client(url=APP_URL, user_name = USER_NAME, dataset_name = DATASET_NAME)

fso_income_metadata = client.get_dataset_metadata()
fso_income_metadata

{'max_ids': 1,
 'rows': 2032543,
 'row_privacy': True,
 'censor_dims': False,
 'columns': {'region': {'private_id': False,
   'nullable': False,
   'max_partition_length': None,
   'max_influenced_partitions': None,
   'max_partition_contributions': None,
   'type': 'int',
   'precision': 32,
   'cardinality': 7,
   'categories': [1, 2, 3, 4, 5, 6, 7]},
  'eco_branch': {'private_id': False,
   'nullable': False,
   'max_partition_length': None,
   'max_influenced_partitions': None,
   'max_partition_contributions': None,
   'type': 'int',
   'precision': 32,
   'cardinality': 72,
   'categories': [8,
    10,
    11,
    13,
    14,
    15,
    16,
    17,
    18,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    31,
    32,
    33,
    35,
    37,
    38,
    41,
    42,
    43,
    45,
    46,
    47,
    49,
    50,
    52,
    53,
    55,
    56,
    58,
    59,
    60,
    61,
    62,
    63,
    64,
    65,
    66,
    68,
    69,
    70,
